# Loading fMRI data & RSA

---
_This code is loosely based on a previous notebook by [Marianne de Heer Kloots](https://mdhk.net/)_

This notebook provides instructions to load fMRI data from the [Pereira](https://www.nature.com/articles/s41467-018-03068-4) dataset and set it up for representational similarity analysis.

Note that this code will show you how to analyse data from only **one participant**, but you should consider all **16 participants** for your projects.

The fMRI recordings from all participants are available in [this](https://drive.google.com/drive/folders/1MjFOyJ9zYwec2bJqL4eNXO4Mlilt0oP8) GG Drive folder by courtesy of the authors of the original paper.

Alright, let's start by loading data from participant M01. Add a shortcut to the M01 folder to your GG Drive, then mount drive so that the folder becomes available in this notebook.

In [ ]:
import numpy as np
from scipy.io import loadmat
from pathlib import Path
from scipy.stats import spearmanr
from sklearn.preprocessing import normalize
import random

In [ ]:
# replace with the path from your GG Drive file system :)

!tar -xf '/content/drive/MyDrive/Pereira/M01.tar'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now, let's load the fMRI recorgings for the condition we're interested in—the original experiment included two additional conditions (`data_180concepts_sentences.mat` and `data_180concepts_wordclouds.mat`), but we do not consider them here.

Note that we're going to work with data that was originally saved in Matlab, so this will create some 'weird' array structures and indexing issues.

In [ ]:
from scipy.io import loadmat
fmri_data = loadmat('M01/data_180concepts_pictures.mat')

In [ ]:
import pandas as pd

# -----------------------------------------------------------
# 1. 180 concepts from the .mat file
# -----------------------------------------------------------
concepts = [str(cell[0]) for cell in fmri_data['keyConcept'].squeeze()]  # list[str]
concept_set = set(concepts)


# -----------------------------------------------------------
# 2.  Load the CSV that contains template sentences
# -----------------------------------------------------------
csv_path = '/content/drive/MyDrive/Masters/4th Sem/UvA/NLP2/Project/Dataset/screen_sentences.csv'
df       = pd.read_csv(csv_path)

csv_concepts = set(df['Concept'].str.strip().str.lower())   # normalise spacing / case
print(csv_concepts)

# -----------------------------------------------------------
# 3.  Compare the two sets
# -----------------------------------------------------------
missing = sorted(concept_set - csv_concepts)   # in .mat list but not in CSV
extra   = sorted(csv_concepts - concept_set)   # in CSV but not in .mat list

print(f"Total concepts in .mat : {len(concept_set)}")
print(f"Total concepts in CSV  : {len(csv_concepts)}\n")

if missing:
    print("Missing concepts (not covered by CSV):")
    for w in missing:
        print("   •", w)
else:
    print("CSV covers all concepts.")

if extra:
    print("\n Extra concepts in CSV (not in .mat list):")
    for w in extra:
        print("   •", w)


{'art', 'feeling', 'pleasure', 'suspect', 'fish', 'bed', 'dig', 'construction', 'dressing', 'crazy', 'business', 'student', 'laugh', 'reaction', 'team', 'ship', 'vacation', 'star', 'experiment', 'dance', 'gun', 'professional', 'movement', 'computer', 'invisible', 'code', 'garbage', 'magic', 'music', 'hurting', 'dessert', 'soul', 'bag', 'land', 'shape', 'skin', 'news', 'taste', 'emotionally', 'charming', 'toy', 'texture', 'dog', 'bird', 'quality', 'religious', 'extremely', 'accomplished', 'angry', 'economy', 'argument', 'prison', 'table', 'light', 'driver', 'dissolve', 'show', 'marriage', 'burn', 'cockroach', 'damage', 'level', 'counting', 'help', 'medication', 'bear', 'picture', 'beat', 'sad', 'poor', 'jungle', 'tried', 'big', 'camera', 'delivery', 'applause', 'disturb', 'trial', 'tree', 'invention', 'protection', 'ball', 'kindness', 'read', 'apartment', 'material', 'mathematical', 'do', 'charity', 'building', 'obligation', 'solution', 'successful', 'relationship', 'flow', 'play', 'roa

The actual fMRI responses for the 180 concepts are stored here:

In [ ]:
fmri_data['examples'].shape

(180, 190042)

Each of the 190042 measures corresponds to a specific voxel, but we're not interested in all of them—we should isolate the ones corresponding to the brain network we're analysing. For the rest of the notebook, we'll assume we're interested in the LH language network.

To isolate its corresponding voxels, we'll take advantage of the information stored in `fmri_data['meta']`, which will allow us to perform the following operations:

1. Map the brain network we're interested in to a number of specific regions of interest (ROIs)
2. Map the ROIs to specific column indexes
3. Use the indexes to slice `fmri_data['examples']`

Let's start by 1. Recall that fMRI data was recorded in multiple brain networks, which are displayed below. The networks that are relevant for the projects are the following:

* Task-positive network (here referred to as multiple-demand system): `MD`
* Default mode network: `DMN`
* Visual network: `visual`
* Left-hemisphere language network: `languageLH`
* Right-hemisphere language network: `languageRH`

In [ ]:
print(fmri_data['meta']['atlases'][0][0][0].shape)
fmri_data['meta']['atlases'][0][0][0] # all the zeros are because of the matlab weird data structures

(15,)


array([array(['aal_1ofN'], dtype='<U8'),
       array(['languageParcelsConservative_aal'], dtype='<U31'),
       array(['languageParcels_aal'], dtype='<U19'),
       array(['semantic_aal'], dtype='<U12'),
       array(['multipleDemand_aal'], dtype='<U18'),
       array(['MD'], dtype='<U2'), array(['DMN'], dtype='<U3'),
       array(['languageLH'], dtype='<U10'),
       array(['languageRH'], dtype='<U10'),
       array(['visual_body'], dtype='<U11'),
       array(['visual_face'], dtype='<U11'),
       array(['visual_object'], dtype='<U13'),
       array(['visual_scene'], dtype='<U12'),
       array(['visual'], dtype='<U6'), array(['gordon'], dtype='<U6')],
      dtype=object)

Each network corresponds to fMRI recordings from multiple ROIs, so now we have to find a way to understand which ROIs should be considered for each network.

The complete list of ROIs is available below.

In [ ]:
print(fmri_data['meta']['rois'][0][0][0].shape)
fmri_data['meta']['rois'][0][0][0]

(15,)


array([array([[array(['Precentral_L'], dtype='<U12')],
              [array(['Precentral_R'], dtype='<U12')],
              [array(['Frontal_Sup_L'], dtype='<U13')],
              [array(['Frontal_Sup_R'], dtype='<U13')],
              [array(['Frontal_Sup_Orb_L'], dtype='<U17')],
              [array(['Frontal_Sup_Orb_R'], dtype='<U17')],
              [array(['Frontal_Mid_L'], dtype='<U13')],
              [array(['Frontal_Mid_R'], dtype='<U13')],
              [array(['Frontal_Mid_Orb_L'], dtype='<U17')],
              [array(['Frontal_Mid_Orb_R'], dtype='<U17')],
              [array(['Frontal_Inf_Oper_L'], dtype='<U18')],
              [array(['Frontal_Inf_Oper_R'], dtype='<U18')],
              [array(['Frontal_Inf_Tri_L'], dtype='<U17')],
              [array(['Frontal_Inf_Tri_R'], dtype='<U17')],
              [array(['Frontal_Inf_Orb_L'], dtype='<U17')],
              [array(['Frontal_Inf_Orb_R'], dtype='<U17')],
              [array(['Rolandic_Oper_L'], dtype='<U15')],
      

Note that the ROIs are grouped into 15 arrays, corresponding to the 15 networks. Therefore, if we want to identify the ROIs corresponding to `languageLH`, which is the eighth network, we just have to consider the ROIs from the eighth array. Let's put things together in a more structured way.

In [ ]:
network_of_interest = 'languageLH'
networks = [network[0] for network in fmri_data['meta']['atlases'][0][0][0]]
nw_index = networks.index(network_of_interest)
rois_of_interest = [roi[0][0] for roi in fmri_data['meta']['rois'][0][0][0][nw_index]]
rois_of_interest

[np.str_('L_PostTemp'),
 np.str_('L_AntTemp'),
 np.str_('L_AngG'),
 np.str_('L_IFG'),
 np.str_('L_MFG'),
 np.str_('L_IFGorb')]

Great! Now we know which ROIs we should consider. Next, we'll have to map them to specific indexes. Note that the mapping, in principle, can be done even without knowing how each ROI is called, but knowing about ROIs will help you make sense of all the for loops we're using.

In [ ]:
nw_columns = fmri_data['meta']['roiColumns'][0][0][0][nw_index]
column_indexes = np.concat([nw_columns[roi][0].flatten()-1 for roi in range(len(nw_columns))], axis=0)
column_indexes.shape

(5716,)

Wondering why we added the -1 in the list comprehension above? That's because indexing starts from 1 in matlab so, if we want to make sure to retrieve the right columns in Python, we have to subtract 1.

⚠️ Note that there's a mistake in how the authors saved data for the visual network, meaning that you'll get 16 ROIs if you look at `fmri_data['meta']['rois']` but only 15 arrays if you look at `fmri_data['meta']['roiColumns']`. In practice, that's not a problem, and you can safely work with the 15 arrays you have :)

Great! Now, one last step. What if we want to isolate the brain responses corresponding to a specific concept, e.g., 'ability'? Concepts are stored in `fmri_data['keyConcept']` so, again, it's just a matter of index mapping.

In [ ]:
print(fmri_data['keyConcept'][:3])

all_concepts = [concept[0][0]for concept in fmri_data['keyConcept']]
ability_index = all_concepts.index('ability')
print(f"Index for 'ability': {ability_index}")

[[array(['ability'], dtype='<U7')]
 [array(['accomplished'], dtype='<U12')]
 [array(['angry'], dtype='<U5')]]
Index for 'ability': 0


Putting all things together, if you want to isolate the brain responses in the `languageLH` network for the concept 'ability', you just have to to the following:

In [ ]:
ability_fmri_data = fmri_data['examples'][ability_index, column_indexes]
ability_fmri_data.shape

(5716,)

If you lived through this index mapping nightmare, here's your reward: A function for doing all we've seen so far altogether! This function takes as input the raw matlap data and the name of the brain network of interest and outputs a `180xn_rois` Numpy array (where 180 is the number of concept words and `n_rois` is the number of ROIs that are relevant for the brain network of interest).

In [ ]:
def get_network_activations(fmri_data_matlab, network_name):

  networks = [atlas[0] for atlas in fmri_data_matlab['meta']['atlases'][0][0][0]]
  idx = networks.index(network_name)

  roi_cols = fmri_data_matlab['meta']['roiColumns'][0][0][0][idx]
  column_indexes = np.concat([roi_cols[roi][0].flatten()-1 for roi in range(len(roi_cols))], axis=0)

  network_responses = fmri_data_matlab['examples'][:, column_indexes]

  return network_responses

In [ ]:
# ---------- 1. Build group-level 180 × V brain matrix -------
import os, tarfile
brain_mats = []

NETWORK      = "languageLH"
SEED         = 123
np.random.seed(SEED)
random.seed(SEED)
drive_root = Path('/content/drive/MyDrive/Pereira')   # adjust if needed
participant_ids = [f"M{idx:02d}" for idx in range(1, 4)]
brain_rdms = []


for pid in participant_ids:
    tar_path = drive_root / f"{pid}.tar"

    if not (Path(pid) / 'data_180concepts_pictures.mat').exists():
        print(f"Extracting {pid}.tar …")
        tarfile.open(tar_path).extractall(path='.')   # untar into current dir
    else:
        print(f"{pid} already extracted ✔")           # skip tar -xf

    fmri_mat = loadmat(f"{pid}/data_180concepts_pictures.mat")

    fmri_data   = get_network_activations(fmri_mat, NETWORK)   # (1080, V)
    rdm = build_rdm(fmri_data)                                  # 180 × 180  (cosine distance)
    brain_rdms.append(rdm)


group_rdm = np.mean(brain_rdms, axis=0)                 # element-wise mean
print("Group RDM shape:", group_rdm.shape)              # (180, 180)

M01 already extracted ✔
dict_keys(['__header__', '__version__', '__globals__', 'labels_task', 'labelsConcept', 'keyConcept', 'labelsPOS', 'keyPOS', 'labelsConcreteness', 'meta', 'examples', 'examples_task', 'tstats_task'])
dimx
dimy
dimz
dimensions
indicesIn3D
colToCoord
coordToCol
voxelsToNeighbours
numberOfNeighbours
roiIDs
nrois
roiColumns
nVoxels
nvoxels
atlases
rois
roiMultimask
M01 loaded : (180, 190042)
Shape of X: (180, 5716)
(180, 180)
M02 already extracted ✔
dict_keys(['__header__', '__version__', '__globals__', 'labels_task', 'labelsConcept', 'keyConcept', 'labelsPOS', 'keyPOS', 'labelsConcreteness', 'meta', 'examples', 'examples_task', 'tstats_task'])
dimx
dimy
dimz
dimensions
indicesIn3D
colToCoord
coordToCol
voxelsToNeighbours
numberOfNeighbours
roiIDs
nrois
roiColumns
nVoxels
nvoxels
atlases
rois
roiMultimask
M02 loaded : (180, 170712)
Shape of X: (180, 4930)
(180, 180)
M03 already extracted ✔
dict_keys(['__header__', '__version__', '__globals__', 'labels_task', 'labelsC

As a last step, let's see how we can construct a representational dissimilarity matrix (RDM) from this data.



In [ ]:
from scipy.spatial.distance import pdist, squareform

def build_rdm(fmri_data):

  # X  = (180, V) activation matrix or embedding matrix
  print("Shape of X:", fmri_data.shape)
  X_norm = normalize(fmri_data, axis=1)  # row-wise L2 normalisation

  # obtain RDM with cosine distances
  rdm = squareform(pdist(X_norm, metric='cosine'))
  print(rdm.shape)

  # select values from the upper triangle
  n = rdm.shape[0]
  rdm_up_tri = rdm[np.triu_indices(n, k=1, m=180)] # k=1 means that we don't want to include values on the diagonal

  return rdm_up_tri

In [ ]:
from scipy.spatial.distance import pdist, squareform

# get fMRI responses for languageLH
langLH_fmri_data = get_network_activations(fmri_data, 'languageLH')
print(langLH_fmri_data.shape)

# obtain RDM with cosine distances
rdm = squareform(pdist(fmri_data))
print(rdm.shape)

# select values from the upper triangle
rdm_up_tri = rdm[np.triu_indices(n=180, k=1, m=180)] # k=1 means that we don't want to include values on the diagonal
rdm_up_tri.shape

(180, 5716)
(180, 180)


(16110,)

Great! The array `rdm_up_tri` is what you should correlate with the corresponding array from model representations—good luck! :)



---



---



---

**Generate BERT Embeddings**


For every concept word, we have 6 example sentences using them. This data is from another study.

Next, we do the following:
1. Generate model embeddings for each concept based on its 6 sentences.
2. Average them to get a single vector per concept.




In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")
model.eval()  # inference mode

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
from collections import defaultdict

concept_to_sentences = defaultdict(list)
concept_embeddings = {}

for _, row in df.iterrows():
    concept = row['Concept']
    sentence = row['Sentence_Screen']
    concept_to_sentences[concept].append(sentence)

# Check lengths
for c, sents in concept_to_sentences.items():
    if len(sents) < 5:
        print(f"{c} has {len(sents)} sentences!")

    sent_embeddings = []

    for sentence in sents:
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)

        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        sent_embeddings.append(cls_embedding)

    concept_embeddings[concept] = np.mean(sent_embeddings, axis=0)

**Create a matrix of shape (180 × D)**

In [ ]:
embedding_matrix = np.vstack([concept_embeddings[c] for c in concepts])  # shape: (180, D)
print("Embedding matrix shape:", embedding_matrix.shape)

**Build model RDM**

In [ ]:
from scipy.spatial.distance import pdist, squareform

def build_rdm(X):
    return squareform(pdist(X, metric='cosine'))  # shape: (180, 180)

model_rdm = build_rdm(embedding_matrix)

model_rdm_vec = model_rdm[np.triu_indices(180, k=1)]